In [1]:
import PIL
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFilter

import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import random
from pascal_voc_writer import Writer
def RandomPaste(origin, img):
    # Resize the pasted image
    width, height = origin.size
    propotion = random.uniform(0.1, 0.7)
    img_width = int(propotion * width)
    img_height = int(propotion * height)
    img = img.resize((img_width, img_height), Image.ANTIALIAS)
    
    # Rotate the pasted image
    rotate_angle = random.randint(0, 360)
    img = img.rotate(rotate_angle, expand=True)
    
    # Crop extra edges
    maxsize = (width / 2, height / 2)
    img.thumbnail(maxsize, Image.ANTIALIAS)
    imageSize = img.size
    imageComponents = img.split()
    rgbImage = Image.new("RGB", imageSize, (0,0,0))
    rgbImage.paste(img, mask=imageComponents[3])
    croppedBox = rgbImage.getbbox()
    img = img.crop(croppedBox)
    
    # Paste image
    r,g,b,a = img.split()
    img_x = int(random.uniform(0, 1) * (width - img_width))
    img_y = int(random.uniform(0, 1) * (height - img_height))
    origin.paste(img, (img_x, img_y), a)
    img_width, img_height = img.size
    
    return (img_x, img_y, img_width, img_height)

In [2]:
def union(a, b):
    x = min(a[0], b[0])
    y = min(a[1], b[1])
    w = max(a[0] + a[2], b[0] + b[2]) - x
    h = max(a[1] + a[3], b[1] + b[3]) - y
    return (x, y, w, h)

In [3]:
def intersection(a, b):
    x = max(a[0], b[0])
    y = max(a[1], b[1])
    w = min(a[0] + a[2], b[0] + b[2]) - x
    h = min(a[1] + a[3], b[1] + b[3]) - y
    if w < 0 or h < 0: 
        return ()
    return (x, y, w, h)

In [4]:
def combine_boxes(boxes, new):
    flag = False
    for i in range(len(boxes) - 1, -1, -1):
        if intersection(boxes[i], new):
            flag = True
            newbox = union(boxes[i], new)
            boxes.remove(boxes[i])
            combine_boxes(boxes, newbox)
            
    if not flag:
        boxes.append(new)
        
    return boxes

In [5]:
def generatePictures(num_pair, num_obj=None, noise_type=None, deviation=False, overlapping=False):
    '''
    num_pair: The number of the pairs generated
    num_obj: The number of the objects pasted, default to 2-3
    noise_type: The type of the noise applied to the pics, default to random
    deviation: Whether apply deviation to the pics, default to False
    '''
    
    bg_path = "./img/background"
    obj_path = "./img/object"
    images_path = "./data/images"
    xml_path = "./data/XML"
    
    for i in range(num_pair):
        num = i+1
        bg_name = random.choice(os.listdir(bg_path))
        print(bg_name)
        background = Image.open(os.path.join(bg_path, bg_name))
        obj = []
        background.save(os.path.join(images_path,str(num)+'.jpg'))
        
        if not num_obj:
            num_obj = random.randint(2, 3);
        for i in range(num_obj):
            obj.append(Image.open(os.path.join(obj_path, random.choice(os.listdir(obj_path)))))

        bndbox = []
        box_loc=[]
        for i in obj:
            (xmin, ymin, xoffset, yoffset) = RandomPaste(background, i)
            box_loc.append((xmin,ymin,xoffset,yoffset))
            bndbox = combine_boxes(bndbox, (xmin, ymin, xoffset, yoffset))

        # Add noise
        
        # Add deviation

        # Adjust Color Temperature

        # Adjust Color Difference

#         plt.figure(figsize=(10, 20))
#         plt.imshow(background)
        background.save(os.path.join(images_path,str(num)+'_A.jpg'))
        
        # Write box to XML
        width, height = background.size
        writer = Writer(str(num)+'.jpg',width,height)
        print(box_loc)
        for i_loc in box_loc :
            writer.addObject('True',i_loc[0],i_loc[1],i_loc[0]+i_loc[2],i_loc[1]+i_loc[3])
        writer.save(os.path.join(xml_path,str(num)+".xml"))
        
        # Change Bounding Box Format
        for b in bndbox:
            box = list(b)
            box[2] += box[0]
            box[3] += box[1]
            b = tuple(box)
            
        # Save Pictures
            
        print(bndbox)

In [6]:
# generatePictures(200)
print("a")

a
